In [ ]:
import pandas as pd
import datetime
import numpy as np
# Testing

# Linkedin Data from Kaggle

In [ ]:
data = pd.read_csv("job postings 2023 24/postings.csv")

In [ ]:
data.columns

In [ ]:
def convert_unix_to_ddmmyyyy(unix_time):
    # Convert from milliseconds to seconds
    unix_time_seconds = unix_time / 1000.0
    
    # Convert to datetime and handle errors
    formatted_dates = pd.to_datetime(unix_time_seconds, unit='s', errors='coerce')
    
    # Format as 'dd-mm-yyyy'
    return formatted_dates.dt.strftime('%d-%m-%Y')

In [ ]:
raw_data = data.copy()

In [ ]:
raw_data['original_listed_time_mod'] = convert_unix_to_ddmmyyyy(raw_data['original_listed_time'])

In [ ]:
raw_data['original_listed_time_mod'] =pd.to_datetime(raw_data['original_listed_time_mod'], format='%d-%m-%Y', errors='coerce')

In [ ]:
raw_data['original_listed_time_mod'].min()

In [ ]:
raw_data['original_listed_time_mod'].max()

# Data Scraped from ScrapingDog API

In [ ]:
import requests

def fetch_all_linkedin_jobs(api_key, fields, geoids, pages):
    url = "https://api.scrapingdog.com/linkedinjobs/"
    all_data = []  # Store data from all requests

    # Iterate over each combination of field, geoid, and page
    for field in fields:
        for geoid in geoids:
            for page in pages:
                params = {
                    "api_key": api_key,
                    "field": field,
                    "geoid": geoid,
                    "page": page
                }
                
                response = requests.get(url, params=params)
                
                if response.status_code == 200:
                    # Append data from each request to the list
                    all_data.extend(response.json())
                else:
                    print(f"Request failed for field: {field}, geoid: {geoid}, page: {page} with status code: {response.status_code}")

    return all_data

# Usage example
api_key = "671da7467fddaf7ca053001a"
fields = ["java", "data science", "machine learning"]
geoids = ["100293800","103736294","102277331"]  # NV, Colorado, California
pages = ["1", "2", "3"]

# data = fetch_all_linkedin_jobs(api_key, fields, geoids, pages)



In [ ]:
if data:
    new_Data = pd.DataFrame(data)
    new_Data.to_csv("261024_Job_Data.csv")

# Data Preprocessing

## Filtering Data

Filtering Function Overview
The filter_data function dynamically filters a DataFrame based on a specified list of columns and corresponding conditions. This flexibility allows for customized data extraction based on various criteria, such as dates, maximum or minimum values, and frequent items.

Function Parameters
df (pd.DataFrame): The input DataFrame containing the data you want to filter.

columns (list): A list of column names to apply filters on. Each column corresponds to a condition in the conditions list.

conditions (list): A list of conditions, where each element is a tuple specifying the operation and the value for filtering. Each tuple matches one of the columns in the columns list.


In [20]:
import pandas as pd

def filter_data(df, columns, conditions):
    """
    Filters a DataFrame based on a list of columns and their conditions.

    Parameters:
    - df (pd.DataFrame): The input DataFrame to filter.
    - columns (list): List of column names to apply filters on.
    - conditions (list): List of conditions where each element is a tuple in the format:
                         (condition, value), for example ('>', '2024-04-20').
                         Each tuple should correspond to a column in the columns list.
    
    Returns:
    - pd.DataFrame: A filtered DataFrame.
    """
    # Loop over each column and its corresponding condition
    for column, (condition, value) in zip(columns, conditions):
        if condition == '>':
            df = df[df[column] > value]
        elif condition == '<':
            df = df[df[column] < value]
        elif condition == '==':
            df = df[df[column] == value]
        elif condition == '>=':
            df = df[df[column] >= value]
        elif condition == '<=':
            df = df[df[column] <= value]
        elif condition == '!=':
            df = df[df[column] != value]
        elif condition == 'max':
            max_value = df[column].max()
            df = df[df[column] == max_value]
        elif condition == 'min':
            min_value = df[column].min()
            df = df[df[column] == min_value]
        elif condition == 'top_n':
            top_n_counts = df[column].value_counts().nlargest(value)
            df = df[df[column].isin(top_n_counts.index)]
        elif condition == 'top_n':
            top_n_counts = df[column].value_counts()#.nlargest(value)
            df = df[df[column].isin(top_n_counts.index)]
        else:
            print(f"Invalid condition '{condition}' for column '{column}'")
    
    return df


## Filtering Scraped Data by Date

In [ ]:
# Load the data
scraped_data = pd.read_csv("C:/Users/DELL/Downloads/261024_Job_Data.csv")

# Convert date column to datetime if filtering by date
scraped_data['job_posting_date'] = pd.to_datetime(scraped_data['job_posting_date'], errors='coerce')

# Specify columns and conditions
columns = ['job_posting_date']
conditions = [('>', '2024-04-20')]

# Apply the filter function
filtered_scraped_data = filter_data(scraped_data, columns, conditions)

# Save the filtered data
filtered_scraped_data.to_csv("filtered_scraped_job_data.csv", index=False)

# Print the filtered data
print(filtered_scraped_data)

## Filtering Current Kaggle Data to check top N job positions

In [4]:
import pandas as pd
kaggledata = pd.read_csv("job postings 2023 24/postings.csv")


In [26]:
def filter_top_n_values(df, column, n):
    # Get the top n most frequent values in the specified column
    top_n_values = df[column].value_counts()#.nlargest(n)
    return top_n_values
columns = ['title']
conditions = [('top_n',50)]
top_15_job_roles = filter_data(kaggledata, columns, conditions)

print(top_15_job_roles)

            job_id                                       company_name  \
19       115639136                                  Shannon Waltchack   
22       133130219                                                NaN   
26       175485704                                               GOYT   
30       229924287  REquipment Durable Medical Equipment and Assis...   
33       258868844                                               ABME   
...            ...                                                ...   
123802  3906263304                                     Synectics Inc.   
123811  3906264238                               CORA Health Services   
123828  3906265264                                    Theoria Medical   
123833  3906265319                                      pepperl+fuchs   
123847  3906267195                    Trelleborg Applied Technologies   

                               title  \
19                        Controller   
22                 Software Engineer   
26 

In [27]:
top_15_job_roles['title'].unique()

array(['Controller', 'Software Engineer', 'Administrative Assistant',
       'Salesperson', 'Registered Nurse', 'Associate Attorney',
       'Sales Representative', 'Customer Service Representative',
       'Staff Accountant', 'Executive Assistant', 'Project Engineer',
       'Project Manager', 'Account Manager', 'Senior Software Engineer',
       'Receptionist', 'Medical Assistant', 'Nurse Practitioner',
       'Accountant', 'Data Analyst', 'Accounting Manager',
       'Sales Associate', 'Sales Specialist', 'Sales Executive',
       'Operations Manager', 'Account Executive', 'Warehouse Associate',
       'Electrical Engineer', 'General Manager', 'Service Technician',
       'Physical Therapist', 'Business Development Manager',
       'Senior Accountant', 'Assistant Manager', 'Maintenance Technician',
       'Manufacturing Engineer', 'Store Manager', 'Financial Analyst',
       'Business Analyst', 'Retail Sales Associate', 'Sales Manager',
       'Assistant Store Manager',
       'Pack

## Data cube

In [5]:
data_cube = kaggledata.pivot_table(
    values=[
        'med_salary', 'max_salary', 'min_salary', 'views', 'applies', 'normalized_salary'
    ],
    index=[
        'location', 'company_name', 'title', 'formatted_work_type', 'remote_allowed', 'formatted_experience_level'
    ],
    aggfunc={
        'med_salary': 'mean',           # Average median salary
        'max_salary': 'max',            # Maximum salary
        'min_salary': 'min',            # Minimum salary
        'views': 'sum',                 # Total views
        'applies': 'sum',               # Total applications
        'normalized_salary': 'mean'     # Average normalized salary
    }
).reset_index()

# Display the resulting data cube
print("Data Cube:")
print(data_cube)

# Optional: Save to CSV for further analysis
data_cube.to_csv('job_data_cube.csv', index=False)

Data Cube:
                    location                    company_name  \
0                 Abbott, TX                      TEKsystems   
1                Abilene, TX  International Rescue Committee   
2                Abilene, TX                         J. Galt   
3                Addison, TX                        Acrisure   
4                Addison, TX                       CBase Inc   
...                      ...                             ...   
9567  Youngstown-Warren area                         J. Galt   
9568           Ypsilanti, MI                      BetterHelp   
9569           Yuba City, CA                         J. Galt   
9570               Yukon, OK                      BetterHelp   
9571                Yuma, AZ                         J. Galt   

                                                  title formatted_work_type  \
0                                     Armis/OT Engineer           Full-time   
1                English Language Instruction Case Aide       

## Dynamic data cube

In [6]:
def dynamic_data_cube(df, company_name=None, title=None, location=None, work_type=None):
    filtered_df = df.copy()
    
    # Apply filters based on user input
    if company_name:
        filtered_df = filtered_df[filtered_df['company_name'] == company_name]
    if title:
        filtered_df = filtered_df[filtered_df['title'] == title]
    if location:
        filtered_df = filtered_df[filtered_df['location'] == location]
    if work_type:
        filtered_df = filtered_df[filtered_df['formatted_work_type'] == work_type]
    
    # Check if filtered DataFrame is not empty
    if filtered_df.empty:
        return "No data found for the specified filters."
    
    # Aggregate metrics for the filtered data
    result = {
        'Total Job Listings': len(filtered_df),
        'Max Salary': filtered_df['max_salary'].max(),
        'Min Salary': filtered_df['min_salary'].min(),
        'Average Median Salary': filtered_df['med_salary'].mean(),
        'Total Views': filtered_df['views'].sum(),
        'Total Applications': filtered_df['applies'].sum(),
        'Average Normalized Salary': filtered_df['normalized_salary'].mean()
    }
    
    return result


In [7]:
print(dynamic_data_cube(kaggledata, company_name="ServiceNow"))

{'Total Job Listings': 57, 'Max Salary': np.float64(316900.0), 'Min Salary': np.float64(69960.0), 'Average Median Salary': np.float64(313100.0), 'Total Views': np.float64(914.0), 'Total Applications': np.float64(95.0), 'Average Normalized Salary': np.float64(187002.08333333334)}


In [8]:
print(dynamic_data_cube(kaggledata, title="Full Stack Java Developer"))

{'Total Job Listings': 7, 'Max Salary': np.float64(125000.0), 'Min Salary': np.float64(110000.0), 'Average Median Salary': nan, 'Total Views': np.float64(123.0), 'Total Applications': np.float64(40.0), 'Average Normalized Salary': np.float64(117500.0)}
